In [ ]:
import os
import sys
import configparser
import datetime as dt
from collections import defaultdict
from datetime import datetime

from pyzotero import zotero

In [ ]:
DATE_FMT = "%Y-%m-%dT%XZ"

# Load config

In [ ]:
config_file = r"config.cfg"
if not os.path.exists(config_file):
    print(f"config file {config_file} does not exist!")
    sys.exit(0)
    
configFilePath = config_file 
configParser = configparser.RawConfigParser()
configParser.read(configFilePath)
library_id = int(configParser.get("zotero-config", "library_id"))
api_key = configParser.get("zotero-config", "api_key")
library_type = configParser.get("zotero-config", "library_type")

# Define functions

In [ ]:
def date_added(item):
    return datetime.strptime(item["data"]["dateAdded"], DATE_FMT)

In [ ]:
def attachment_is_pdf(_child):
    return (
        _child["data"]["itemType"] == "attachment"
        and _child["data"]["contentType"] == "application/pdf"
    )

In [ ]:
def get_items_with_duplicate_pdf(_items):
    _items_duplicate_attach = []
    _pdf_attachments = defaultdict(list)
    for _item in _items:
        key = _item["key"]
        cs = zot.children(key)
        for c in cs:
            if attachment_is_pdf(c):
                _pdf_attachments[key].append(c["data"]["filename"])

        if len(_pdf_attachments[key]) > 1:
            _items_duplicate_attach.append(_item)

    return _items_duplicate_attach, _pdf_attachments

In [ ]:
def retrieve_data():
    print("Retrieving Library...")
    zot = zotero.Zotero(library_id, library_type, api_key)
    lib_items = zot.everything(zot.top())
    T = dt.datetime.now()
    print(f"Done at {T.hour}:{T.minute}:{T.second}")
    return lib_items

# Delete own attachments 
<a id='del-attach'></a>

Important when merging duplicate items. See explanations in 
[this cell](merge_duplicate_items.ipynb#del-attach).



In [ ]:
DELETE_OWN_ATTACHMENTS = True

# Delete Tags 

Zotero creates special tags to mark duplicates (`duplicate-citation-key`).
Set this to `True` to remove them in this [this cell](remove_duplicate_attachments.ipynb#tags).

In [ ]:
DELETE_TAGS = False  # @todo: check if this is necessary